In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm import tqdm

from shapely.geometry import Point
from shapely.ops import unary_union

Update the equity index values from 2024 to 2025

In [20]:
df_equity_2025 = pd.read_excel('../ct-data/equity-census-2025.xlsx')
gdf_equity_map_2024 = gpd.read_file('../ct-data/equity-2024-layer.geo.json')

In [21]:
# Force both keys to the same string format with 2 decimal places
df_equity_2025["Census tract"] = df_equity_2025["Census tract"].astype(float).map("{:.2f}".format)
gdf_equity_map_2024["ctuid"] = gdf_equity_map_2024["ctuid"].astype(float).map("{:.2f}".format)

# Rename columns for consistency
df_equity_2025.rename(columns={"Census tract": "ctuid"}, inplace=True)
gdf_equity_map_2024 = gdf_equity_map_2024.rename(columns={"Equity Index": "Equity Index 2024"})

# Select and rename columns to merge
columns_to_merge = {
    "Normalized Equity Score": "Equity Index",
    "Eviction rate": "%Evic",
    "Unemployment rate": "%Unemp",
    "  % No certificate, diploma or degree": "%NoEdu",
    "Gov transfer": "%IncomeGT"
}

df_to_merge = df_equity_2025[list(columns_to_merge.keys()) + ["ctuid"]]
df_to_merge = df_to_merge.rename(columns=columns_to_merge)

# Merge data frames on ctuid and save
gdf_equity_map_2025 = gdf_equity_map_2024.merge(df_to_merge, on="ctuid", how="left")

gdf_equity_map_2025.to_file("../ct-data/equity-2025-layer.geo.json", driver="GeoJSON")

Identify the breakpoints for quintiles

In [22]:
gdf_ct = gpd.read_file('../ct-data/equity-2025-layer.geo.json')

# Calculate quintile breakpoints
quintiles = np.quantile(gdf_ct['Equity Index'].dropna(), [0, 0.2, 0.4, 0.6, 0.8, 1])
print("Quintile breakpoints:", quintiles)

Quintile breakpoints: [0.         0.2985527  0.37396513 0.43815575 0.52905651 1.        ]


In [23]:
gdf_ct.columns

Index(['ctuid', 'Own_count', 'Rent_count', 'Unknown_count', 'Total_count',
       'Equity Index 2024', 'Popu_2021', 'PopuDenPerKM', 'Immigrant%', 'VM%',
       '1-ParentFam%', 'MBM%', 'LIM%', 'Neet%', '%CHN', '%Affordable', '%ofWP',
       'ShortTerm%', 'Equity Index', '%Evic', '%Unemp', '%NoEdu', '%IncomeGT',
       'geometry'],
      dtype='object')

In [ ]:
# CODE HERE


Count the number of locations for each census tract as well as an 800m buffer. Further, tag each census tract by the census division

In [5]:
# Read data
gdf_locations = gpd.read_file('../joined-data/simplified_matches_4326_full.geo.json')
gdf_ct = gpd.read_file('../ct-data/equity-2025-layer.geo.json')
gdf_gta = gpd.read_file('../ct-data/GTA_CD.gpkg').to_crs(3857)
gdf_gta = gdf_gta[gdf_gta['CDNAME'].isin(['Toronto', 'Peel', 'York'])]

# Change CRS to 3857 for accurate distance calculations
gdf_locations = gdf_locations.to_crs(epsg=3857)
gdf_ct = gdf_ct.to_crs(epsg=3857)
gdf_gta = gdf_gta.to_crs(epsg=3857)

In [6]:
def get_region_with_max_overlap(tract, gta):
    overlaps = gta[gta.intersects(tract.geometry)]
    if not overlaps.empty:
        overlaps['overlap_area'] = overlaps.geometry.intersection(tract.geometry).area
        return overlaps.loc[overlaps['overlap_area'].idxmax(), 'CDNAME']
    return None

gdf_ct['region'] = gdf_ct.apply(lambda row: get_region_with_max_overlap(row, gdf_gta), axis=1)

In [7]:
# Initialize new columns
for col in ['own_count', 'rent_count', 'unknown_count', 'total_count',
            'own_count_800m', 'rent_count_800m', 'unknown_count_800m', 'total_count_800m']:
    gdf_ct[col] = 0

# Iterate over census tracts
for idx, tract in tqdm(gdf_ct.iterrows(), total=len(gdf_ct)):
    # Locations within the census tract
    locations_in_tract = gdf_locations[gdf_locations.within(tract.geometry)]
    
    # Count tenure categories
    gdf_ct.at[idx, 'own_count'] = (locations_in_tract['Tenure'] == 'Own').sum()
    gdf_ct.at[idx, 'rent_count'] = (locations_in_tract['Tenure'] == 'Rent').sum()
    gdf_ct.at[idx, 'unknown_count'] = (locations_in_tract['Tenure'] == 'Unknown').sum()
    gdf_ct.at[idx, 'total_count'] = len(locations_in_tract)

    # Buffer geometry for 800m radius around the entire tract
    buffer_800m = tract.geometry.buffer(800)
    locations_in_buffer = gdf_locations[gdf_locations.geometry.intersects(buffer_800m)]

    # centroid = tract.geometry.centroid
    # buffer_800m = centroid.buffer(800)
    # locations_in_buffer = gdf_locations[gdf_locations.geometry.intersects(buffer_800m)]
    
    # Count tenure categories within 800m
    gdf_ct.at[idx, 'own_count_800m'] = (locations_in_buffer['Tenure'] == 'Own').sum()
    gdf_ct.at[idx, 'rent_count_800m'] = (locations_in_buffer['Tenure'] == 'Rent').sum()
    gdf_ct.at[idx, 'unknown_count_800m'] = (locations_in_buffer['Tenure'] == 'Unknown').sum()
    gdf_ct.at[idx, 'total_count_800m'] = len(locations_in_buffer)

# Drop the original columns
gdf_ct = gdf_ct.drop(columns=['Own_count', 'Rent_count', 'Unknown_count', 'Total_count'])

  0%|          | 0/1065 [00:00<?, ?it/s]

100%|██████████| 1065/1065 [00:02<00:00, 371.10it/s]


In [8]:
# Revert CRS back to 4326 before saving
gdf_ct = gdf_ct.to_crs(epsg=4326)

# Save updated GeoDataFrame
gdf_ct.to_file('../ct-data/ct-data-all.geo.json', driver='GeoJSON')